In [2]:
class Session:
    def __init__(self, query, region, urls, clicks):
        self.query = query
        self.region = region
        self.urls = [0]
        self.urls.extend(urls)
        self.clicks = clicks   

In [3]:
def parseSessions(fileInput):
    sessions = []
    for line in fileInput:
        parts = line.strip().split("\t")
        query = parts[2]
        region = parts[3]
        count = int(parts[4])
        urls = parts[5:5 + count]
        clicks = map(int, parts[5 + count:])
        sessions.append(Session(query, region, urls, clicks))
    return sessions

In [4]:
filename = "sessions_1000.txt"
fileInput = open(filename)
sessions = parseSessions(fileInput)

In [22]:
class Statistics:
    def __init__(self, query, url):
        self.query = query
        self.url = url
        self.a_D = 0
        self.a_N = 0
        self.s_N = 0
        self.rel = None

In [79]:
def DBN(sessions):
    docs = {}
    for session in sessions:
        query = session.query
        urls = session.urls
        clicks = session.clicks
        for i in range(1, max(clicks) + 1):
            pair = (query, urls[i])
            doc = docs.get(pair, Statistics(pair[0], pair[1]))
            doc.a_D += 1
            docs[pair] = doc
        for i in clicks:
            pair = (query, urls[i])
            doc[pair].a_N += 1
        pair = (query, urls[clicks[-1]])
        doc[pair].s_N += 1
    docs = docs.values()
    for doc in docs:
        doc.rel = (float(doc.a_N + 1) / (doc.a_D + 2)) * (float(doc.s_N + 1) / (doc.a_N + 2))
    return docs

In [80]:
dbns = DBN(sessions)

In [94]:
queryWeather = filter(lambda x: x[0][0] == 'погода', dbns.items())
queryWeather = sorted(queryWeather, cmp = lambda x, y: (x[1].rel - y[1].rel) > 0)
for item in queryWeather:
    print item[0][1], item[1].rel

http://www.gismeteo.ru/city/daily/5039/ 0.444444444444
weather 0.166666666667
